# Model tunning
**Finding best hyperparameters and properly setups for Model**

I

In [2]:
# Setup

import numpy as np
import pandas as pd

from pkg import utils
from pkg import tuning

import lazypredict
from lazypredict.Supervised import LazyRegressor
import numpy as np
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV


In [3]:
# Importing data

parent = 'data'
filename = 'featured.csv'

path = utils.get_path(parent, filename)

df = pd.read_csv(path, index_col=0)

In [4]:
seed = 123

lazying = False

pd.set_option('display.max_columns', None)
df.head()

,RENDA_ESPERADA,AVAL_SATISFACAO,VL_LIMITE_CREDITO_INICIAL_BANCO,VL_FINANCEIRO_BCB,VL_OPERACAO_CARTAO_CREDITO,VL_LIMITE_CREDITO_BCB,PONTUACAO_DESEMPENHO,TEMPO_EXISTENCIA_ONLINE,AVALIACAO_RISCO,SCORE_INTERNO,TIPO_RELACIONAMENTO_1,TIPO_RELACIONAMENTO_2,TIPO_RELACIONAMENTO_3,TIPO_RELACIONAMENTO_4,ESTADO_LOJA_1,ESTADO_LOJA_2,ESTADO_LOJA_3,ESTADO_LOJA_4,ESTADO_LOJA_5,ESTADO_LOJA_6,ESTADO_LOJA_7,ESTADO_LOJA_8,ESTADO_LOJA_9,ESTADO_LOJA_10,ESTADO_LOJA_11,ESTADO_LOJA_12,ESTADO_LOJA_13,ESTADO_LOJA_14,ESTADO_LOJA_15,ESTADO_LOJA_16,ESTADO_LOJA_17,ESTADO_LOJA_18,ESTADO_LOJA_19,ESTADO_LOJA_20,ESTADO_LOJA_21,ESTADO_LOJA_22,ESTADO_LOJA_23,ESTADO_LOJA_24,ESTADO_LOJA_25,ESTADO_LOJA_26,CATEGORIA_PRODUTO_1,CATEGORIA_PRODUTO_2,CATEGORIA_PRODUTO_3,CATEGORIA_PRODUTO_4,CATEGORIA_PRODUTO_5,ESTRATEGIA_VENDAS_1,ESTRATEGIA_VENDAS_2,REGIAO_LOJA_1,REGIAO_LOJA_2,REGIAO_LOJA_3,REGIAO_LOJA_4,RENDA
ID_LOJA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
922686,0.30,0.89,-0.42,0.36,1.96,0.36,1.39,0.13,0.59,-0.34,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-0.34
938287,-0.03,-1.76,-0.56,-0.42,-0.38,-0.36,-1.19,-1.21,0.59,1.41,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,-0.37
939120,0.03,-0.43,-0.69,-0.43,-0.36,-0.33,0.49,-0.76,0.59,-0.98,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0.35
933554,-0.03,0.23,0.94,-0.18,-0.16,-0.34,0.02,-0.23,-1.22,0.37,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0.04
948294,0.13,2.22,-0.42,-0.38,-0.28,-0.17,1.39,-0.50,0.59,-0.56,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0.50


In [5]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=seed)


In [6]:
if lazying == True:

    highmem_regressors = ["GammaRegressor", "GaussianProcessRegressor", "KernelRidge", "QuantileRegressor"]

    nonmem_reg = [reg for reg in lazypredict.Supervised.REGRESSORS if reg[0] not in highmem_regressors]
    highmem_reg = [reg for reg in lazypredict.Supervised.REGRESSORS if reg[0] in highmem_regressors]

    reg = LazyRegressor(regressors=nonmem_reg,verbose=0,ignore_warnings=False,custom_metric=None)
    models,predictions = reg.fit(X_train, X_test, y_train, y_test)

    print(models)

In [7]:
reg = XGBRegressor(max_depth=1)#100)
reg.fit(X_train, y_train)
y_hat = reg.predict(X_test)

from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_hat)

print(r2)

0.2508323754483389


In [25]:
# Setting default parameters

actual_par = {'max_depth':6, 
            'min_child_weight':1, 
            'gamma':0.0, 
            'subsample':1,
            'colsample_bytree':1,
            'reg_lambda':1,
            'reg_alpha':0}


param_test1 = {'max_depth':range(2,10),
              'min_child_weight':range(0,10)}

param_test2 = {'gamma':[i/10.0 for i in range(0,5)]}

param_test3 = {'subsample':[i/10.0 for i in range(6,11)],
              'colsample_bytree':[i/10.0 for i in range(6,11)]}

param_test4 = {'reg_alpha':[0, 1e-5, 1e-2, 0.1, 1, 100],
               'reg_lambda':range(1,10)}

params = [param_test1,param_test2, param_test3, param_test4]

for par in params:
    reg = XGBRegressor(max_depth=actual_par['max_depth'],
                       min_child_weight=actual_par['min_child_weight'],
                       gamma=actual_par['gamma'],
                       subsample=actual_par['subsample'],
                       colsample_bytree=actual_par['colsample_bytree'],
                       reg_lambda=actual_par['reg_lambda'],
                       reg_alpha=actual_par['reg_alpha'], seed=seed)
    
    grid = GridSearchCV(estimator=reg, param_grid=par, scoring='r2')
    grid.fit(X_train, y_train)
    print('best parameters:', grid.best_params_)
    print('best r2 score', grid.best_score_)

    for item in grid.best_params_:
        for par_ac in actual_par:
            if par_ac == item:
                actual_par[par_ac] = grid.best_params_[par_ac]

best parameters: {'max_depth': 5, 'min_child_weight': 4}
best r2 score 0.5901055485417961
best parameters: {'gamma': 0.0}
best r2 score 0.5901055485417961
best parameters: {'colsample_bytree': 0.7, 'subsample': 1.0}
best r2 score 0.5956983361472755
best parameters: {'reg_alpha': 0, 'reg_lambda': 9}
best r2 score 0.5973882991921714


In [26]:
print(actual_par)

{'max_depth': 5, 'min_child_weight': 4, 'gamma': 0.0, 'subsample': 1.0, 'colsample_bytree': 0.7, 'reg_lambda': 9, 'reg_alpha': 0}
